# HGEN HW5



In [1]:
#pip install cvxpy
import cvxpy as cp 
import numpy as np
#pip install cvxopt
from cvxopt import matrix, solvers
from sklearn.linear_model import Lasso

## Problem 1 Dantzig Selector

### PART 1
Recast the Dantzig Selector into Linear Programming


Recall from the lecture notes, A LP problem is an optimization problem of the form: 

Minimize:
$$ c^T x $$

Subject to:
$$ Ax = b $$
$$ x\geq 0 $$

#### Linear Programming Formulation Using Auxiliary Variables

Use auxiliary variables \(z_j\), where:
$$ z_j \geq \beta_j \quad \text{and} \quad z_j \geq -\beta_j $$

##### Objective:
Minimize the sum of the auxiliary variables:
$$ \sum_{j} z_j $$

#####  Subject to:
For each \( j \):
$$ z_j \geq \beta_j \quad \text{and} \quad z_j \geq -\beta_j $$

For each \( i \):
$$ -\lambda \leq \sum_{j} X_{ij}^T (X\beta - y)_j \leq \lambda $$





In [2]:
# Simulate data (so we can test all our methods)
np.random.seed(1)
n = 200  # number of observations
p = 600  # number of predictors
X = np.random.randn(n, p)
beta_true = np.random.randn(p) * (np.random.rand(p) < 0.1)  # sparse true coefficients
y = X @ beta_true + np.random.randn(n)  # generate response

In [3]:
# Use package cvxpy to solve this convex problem
# Dimensions
lambda_val =1
# Define variables
beta = cp.Variable(p)
z = cp.Variable(p, nonneg=True)  # Auxiliary variable for the L1 norm

# Objective function
objective = cp.Minimize(cp.sum(z))
 
# Constraints 
constraints = [z >= beta, z >= -beta]  # Enforces z_j >= |beta_j|

# Constructing constraint for ||X^T(X*beta - y)||_infty <= lambda
# (Python has the package for this norm constraints so we don't need to translate it into inequality as 
# show in part a)
r = X @ beta - y
XTr = X.T @ r
constraints += [cp.norm(XTr, 'inf') <= lambda_val]

# Problem setup for Dantzig Selector
problem_Dantzig = cp.Problem(objective, constraints)

# Solve the problem
problem_Dantzig.solve()

beta_Dantzig=beta.value
# Print the results
print("Status:", problem_Dantzig.status)
print("Optimal value:", problem_Dantzig.value)
print("Optimal beta:", beta_Dantzig)


/Users/linyu/opt/anaconda3/envs/pycourse/lib/python3.8/site-packages/cvxpy/reductions/solvers/solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)


Status: optimal
Optimal value: 59.073695270978945
Optimal beta: [-9.79236430e-13  1.06120654e-01  2.81761710e-14  2.97815762e-12
  1.89021635e-13  9.29757957e-12 -7.15677921e-04  2.90091530e-01
  5.89827434e-11  7.30840515e-13 -9.36744778e-02  3.63614869e-12
 -1.08110297e-12 -1.13556065e-12  1.64967363e-11 -7.38431161e-13
 -1.03792108e-11  1.08475191e-12 -6.51350219e-02  2.59412757e-01
 -6.28947233e-12  1.28766663e-11  3.64014800e-12 -3.80939429e-12
 -7.01912726e-13  1.28814187e-12  1.60288430e-12 -4.10233200e-14
  1.46811680e-02 -1.11011343e-01  2.31269603e-12 -8.80244061e-14
 -3.46466452e-11 -5.98967225e-13 -1.60152320e-01 -1.26946247e-12
  1.62671422e-13 -1.21222249e-12 -2.35587204e-12  2.99872583e-12
 -3.30903423e-12  2.39442090e-01 -2.38153790e-11 -3.84108792e-12
 -5.03968885e-02 -4.88432317e-13  7.89258445e-01  3.38708512e-12
  2.95404424e-11 -3.02302148e-12 -6.14176719e-12 -1.09358589e-11
  3.37104066e-01  1.94247687e-03  3.54779340e-12 -1.11895153e-01
  6.78105022e-12 -3.622074

## Question 2 Lasso

#### Solve Lasso with cvxpy package

In [4]:

# Set up Lasso problem
lambda_lasso = 1.0
beta = cp.Variable(p)
loss = cp.sum_squares(y - X @ beta)
regularization = lambda_lasso * cp.norm1(beta)
objective = cp.Minimize(loss + regularization)
problem_lasso = cp.Problem(objective)

# Solve the problem
problem_lasso.solve()

# Results
lasso_beta_cvxpy = beta.value


In [5]:
# Print the results for Lasso
print("Status:", problem_lasso.status)
print("Optimal value:", problem_lasso.value)
print("Optimal beta:", lasso_beta_cvxpy)

Status: optimal
Optimal value: 59.781247352119514
Optimal beta: [-4.21953950e-05  1.50399884e-01  2.66853853e-06  4.75087656e-05
 -1.46634518e-05 -3.83403400e-05 -1.57331635e-02  2.57258512e-01
 -1.16032092e-05  4.79353654e-05 -1.03724954e-01  3.76930465e-05
 -5.15254538e-05  1.59674444e-05 -2.51941772e-05  2.13687380e-05
  1.51527447e-05  5.08667096e-06 -3.37838937e-02  2.45847663e-01
 -7.10714835e-05 -3.93948035e-05 -1.31933571e-05  5.80698539e-06
  1.27814187e-05 -3.69511036e-05  5.86556954e-05  1.95747301e-05
  4.18814978e-02 -9.17713407e-02  1.41402023e-05 -2.86323380e-05
  5.06420340e-06  1.65076468e-05 -1.48428656e-01  3.01078126e-05
 -8.45666652e-05 -4.20996883e-05 -4.26249981e-06  1.62222482e-05
  4.44332672e-05  2.15400137e-01 -3.70371804e-02 -2.36826702e-05
 -6.18584432e-02 -3.07562061e-05  8.24931360e-01 -4.61190133e-06
 -1.92606633e-05 -7.98526534e-08 -3.93952447e-05  1.14541646e-04
  3.44395920e-01  8.93424156e-03 -4.72344651e-05 -1.04403284e-01
 -3.13042772e-05  4.578864

#### Benchmark lasso solver from cvxpy package with Lasso from sklearn.linear_model 

In [6]:
from sklearn.linear_model import Lasso

lambda_lasso = 1.0
lasso_model = Lasso(alpha=lambda_lasso, fit_intercept=False)
lasso_model.fit(X, y)

# Retrieve the coefficients
lasso_beta_sklearn = lasso_model.coef_

In [7]:
# Function to calculate MSE
def mse(y_true, y_pred):
    return ((y_true - y_pred) ** 2).mean()

# Calculate predictions from sklearn.linear_model
y_preds_sklearn = X @ lasso_beta_sklearn

# Calculate predictions from cvxpy package 
y_preds_cvxpy = X @ lasso_beta_cvxpy

# Calculate MSE
print("MSE with cvxpy Lasso:", mse(y, y_preds_cvxpy))
print("MSE with sklearn Lasso:", mse(y, y_preds_sklearn))


MSE with cvxpy Lasso: 0.0010563906518943377
MSE with sklearn Lasso: 31.4434067353365


#### Benchmark Dantzig selector

In [8]:
y_preds_Dantzig=X @ beta_Dantzig
print("MSE with cvxpy Dantzig:", mse(y,y_preds_Dantzig))

MSE with cvxpy Dantzig: 0.0057335071418471405


## Question 3 Markowitz portfolio optimization

In [9]:
from cvxopt import matrix, solvers

# Set up the problem parameters
mu_R = np.array([0.05, 0.06, 0.07])  # Expected returns of the assets
Sigma_R = np.array([[0.1, 0.01, 0.01],
                    [0.01, 0.1, 0.01],
                    [0.01, 0.01, 0.1]])  # Covariance matrix of asset returns
l = 0.06  # Minimum acceptable return

# Define the number of assets
n = len(mu_R)

# Quadratic programming matrices
P = matrix(Sigma_R)  # Covariance matrix as a CVXOPT matrix for quadratic objective
q = matrix(np.zeros(n))  # Coefficient vector for the linear term in the objective (zero since we minimize variance)

# Inequality constraints Gx <= h to enforce minimum return and non-negative weights
G = matrix(np.vstack((-mu_R, -np.eye(n))))  # Negative sign because we want returns to be at least 'l' and weights non-negative
h = matrix(np.hstack((-l, np.zeros(n))))  # Right-hand side vector, first entry for minimum return, others for non-negativity

# Equality constraint Ax = b to ensure that the sum of weights equals 1
A = matrix(1.0, (1, n))  # Row matrix of ones (length n)
b = matrix(1.0)  # Scalar value 1 in a CVXOPT matrix

# Solver settings
solvers.options['show_progress'] = False  # Disable solver progress output for cleaner output

# Solve the quadratic programming problem
solution = solvers.qp(P, q, G, h, A, b)

# Output the results
if solution['status'] == 'optimal':
    weights = np.array(solution['x']).flatten()  # Extract and flatten the weight vector
    print("Optimized weights:", weights)
else:
    print("No optimal solution found.")


Optimized weights: [0.33285245 0.33333333 0.33381422]
